In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit_metal import Dict, designs, MetalGUI
from ThreeWaysCapacitor import TriCapacitor

# Design

In [3]:
capdesign = designs.DesignPlanar(overwrite_enabled=True)

In [4]:
gui = MetalGUI(capdesign)

In [5]:
capdesign.variables['cpw_width']='20um'
capdesign.variables['cpw_gap']='12um'

In [6]:
from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital

CapNInterdigital.get_template_options(capdesign)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'north_width': '10um',
 'north_gap': '6um',
 'south_width': '10um',
 'south_gap': '6um',
 'cap_width': '10um',
 'cap_gap': '6um',
 'cap_gap_ground': '6um',
 'finger_length': '20um',
 'finger_count': '5',
 'cap_distance': '50um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False}

In [24]:
cap_opt = Dict(
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='5um',
    cap_gap_ground='12um',
    finger_count='3',
    cap_distance='1um',
    finger_length='55um'
)
cap1 = CapNInterdigital(capdesign, 'CAP1', options=cap_opt)

gui.rebuild()
gui.zoom_on_components(['CAP1'])

In [25]:
cap1

name:    CAP1
class:   CapNInterdigital      
options: 
  'pos_x'             : '0.0um',                      
  'pos_y'             : '0.0um',                      
  'orientation'       : '0.0',                        
  'chip'              : 'main',                       
  'layer'             : '1',                          
  'north_width'       : '20um',                       
  'north_gap'         : '12um',                       
  'south_width'       : '20um',                       
  'south_gap'         : '12um',                       
  'cap_width'         : '5um',                        
  'cap_gap'           : '6um',                        
  'cap_gap_ground'    : '12um',                       
  'finger_length'     : '55um',                       
  'finger_count'      : '3',                          
  'cap_distance'      : '1um',                        
  'hfss_wire_bonds'   : False,                        
  'q3d_wire_bonds'    : False,                        
module:  

# Q3D

In [12]:
from qiskit_metal import renderers

In [13]:
ansys = capdesign.renderers.q3d

In [14]:
ansys.connect_ansys()

INFO 11:26PM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:26PM [load_ansys_project]: 	Opened Ansys App
INFO 11:26PM [load_ansys_project]: 	Opened Ansys Desktop v2023.1.0
INFO 11:26PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Documents/Ansoft/
	Project:   Project9
INFO 11:26PM [connect_design]: 	Opened active design
	Design:    Sweep_Capacitance [Solution type: Q3D]
INFO 11:26PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:26PM [connect]: 	Connected to project "Project9" and design "Sweep_Capacitance" 😀 



In [15]:
ansys.activate_q3d_design('Coupling capacitor')

11:28PM 37s WARNING [activate_q3d_design]: This method is deprecated. Change your scripts to use activate_ansys_design()
11:28PM 37s WARNING [activate_ansys_design]: The design_name=Coupling capacitor was not in active project.  Designs in active project are: 
['Sweep_Capacitance', '3ways_capacitor'].  A new design will be added to the project.  
INFO 11:28PM [connect_design]: 	Opened active design
	Design:    Coupling capacitor [Solution type: Q3D]
WARNING 11:28PM [connect_setup]: 	No design setup detected.
WARNING 11:28PM [connect_setup]: 	Creating Q3D default setup.
INFO 11:28PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)


In [16]:
ansys.render_design()

In [18]:
ansys.add_q3d_setup(
    name='capacitance',
    freq_ghz=7.2,
    min_passes=2,
    max_passes=15,
    percent_error=0.2
)

In [19]:
ansys.analyze_setup('capacitance')

INFO 11:32PM [get_setup]: 	Opened setup `capacitance`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:32PM [analyze]: Analyzing setup capacitance


In [20]:
ansys.get_capacitance_matrix()

INFO 11:35PM [get_matrix]: Exporting matrix data to (D:\Users\e196203\AppData\Local\Temp\2\tmp45kp2cph.txt, C, , capacitance:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 7200000000, Maxwell, 1, False


(                   cap_body_0_CAP1  cap_body_1_CAP1  ground_main_plane
 cap_body_0_CAP1           11.03632         -3.14082           -7.76969
 cap_body_1_CAP1           -3.14082          7.93900           -4.71633
 ground_main_plane         -7.76969         -4.71633           43.68229,
 'fF')

# Sweep

In [21]:
from qiskit_metal.analyses.sweep_and_optimize.sweeping import Sweeping

sweep = Sweeping(capdesign)

In [22]:
setup_args=Dict(
    freq_ghz=7.2,
    min_passes=2,
    max_passes=15,
    percent_error=0.2
)

all_sweep, retorn_code = sweep.sweep_one_option_get_capacitance_matrix(
    qcomp_name='CAP1',
    option_name='finger_length',
    option_sweep=['35um','40um','45um','50um','55um','60um'],
    qcomp_render=['CAP1'],
    endcaps_render=[],
    setup_args=setup_args
)

INFO 11:46PM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:46PM [load_ansys_project]: 	Opened Ansys App
INFO 11:46PM [load_ansys_project]: 	Opened Ansys Desktop v2023.1.0
INFO 11:46PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Documents/Ansoft/
	Project:   Project9
INFO 11:46PM [connect_design]: 	Opened active design
	Design:    Coupling capacitor [Solution type: Q3D]
INFO 11:46PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:46PM [connect]: 	Connected to project "Project9" and design "Coupling capacitor" 😀 

INFO 11:46PM [connect_design]: 	Opened active design
	Design:    Sweep_Capacitance [Solution type: Q3D]
INFO 11:46PM [get_setup]: 	Opened setup `Sweep_q3d_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:46PM [get_setup]: 	Opened setup `Sweep_q3d_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:46PM [analyze]: Analyzing setup Sweep_q3d_setup
INFO 11:49PM [get_matrix]: Exporting matrix data 

In [23]:
all_sweep

{'35um': {'option_name': 'finger_length',
  'convergence_target': 0.2,
  'convergence_current': 0.080852,
  'min_number_passes': 2,
  'is_convergence': True,
  'capacitance': (                   cap_body_0_CAP1  cap_body_1_CAP1  ground_main_plane
   cap_body_0_CAP1           12.29852         -3.63051           -8.51686
   cap_body_1_CAP1           -3.63051          8.67653           -4.95162
   ground_main_plane         -8.51686         -4.95162           44.40748,
   'fF'),
  'convergence_data':       # Triangle   Delta %
  Pass                      
  1             68       NaN
  2            212  5.818100
  3            638  6.518000
  4           1578  1.547500
  5           1936  0.655400
  6           2386  0.439820
  7           3042  0.479960
  8           3952  0.338200
  9           5018  0.159060
  10          6642  0.499120
  11          8422  0.205290
  12         10758  0.090337
  13         13638  0.080852},
 '40um': {'option_name': 'finger_length',
  'convergence_target

In [26]:
ansys.disconnect_ansys()

In [27]:
gui.main_window.close()

True